# Importing of Libraries for Data Cleaning

In [77]:
import numpy as np
import pandas as pd

# Data Cleaning

## Conditions of Data Cleaning

We have identified 3 conditions for match to be considered as valid.

1. Match has to reach at least 16 rounds (Minimum rounds to win is 16) to be considered otherwise, there is abandonment or potential throwing (Intentionally Losing) of the match.

2. For each round, a team's spending cannot exceed 44500 (Maximum value of equipment per person is 8900) and cannot be less than 1000 as everyone starts with a pistol worth 200 dollars.

3. Matches should be played on maps that are within the competitive map pool (Cache, Vertigo, Mirage, Dust2, Cobblestone, Inferno, Overpass, Nuke, Train). This is to remove any showmatches where teams consists of random / non-professional players.

In [ ]:
data = pd.read_csv("/data/economy.csv")

In [79]:
data.shape

(43234, 99)

In [80]:
data.head()

,date,match_id,event_id,team_1,team_2,best_of,_map,t1_start,t2_start,1_t1,...,21_winner,22_winner,23_winner,24_winner,25_winner,26_winner,27_winner,28_winner,29_winner,30_winner
0,2020-03-01,2339402,4901,G2,Natus Vincere,5,Nuke,t,ct,4350.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-01,2339402,4901,G2,Natus Vincere,5,Dust2,ct,t,3900.0,...,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,NaN
2,2020-03-01,2339402,4901,G2,Natus Vincere,5,Mirage,t,ct,4150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Dust2,t,ct,4150.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Nuke,ct,t,4200.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Condition 1
Ensuring that every match entry reaches 16 rounds and removing any invalid matches that do not reach 16 rounds.

In [81]:
datacopy = data #Create a copy of data to conduct data cleaning on

In [82]:
temphead = list(datacopy.columns.values) #Get names of columns
splitHead = temphead[9:99] #splitHead contains the column names of team 1's spending, team 2's spending and the winning team from Round 1 to 30

countlist = [] #Contains the number of entries for each column

for column in splitHead:
  countlist.append(np.sum(datacopy[column].count())) #Count the number of entries for each column and adds them to countList

print(countlist[0:30]) #Number of match entries of team 1's spending from Round 1 to 30
print(countlist[30:60]) #Number of match entries of team 2's spending from Round 1 to 30
print(countlist[60:90]) #Number of match entries of winning team from Round 1 to 30

[43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43083, 42459, 41261, 39701, 37666, 35261, 32366, 29394, 25993, 22532, 18948, 15287, 11623, 7899]
[43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43083, 42459, 41261, 39701, 37666, 35261, 32366, 29394, 25993, 22532, 18948, 15287, 11623, 7899]
[43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43234, 43083, 42459, 41261, 39701, 37666, 35261, 32366, 29394, 25993, 22532, 18948, 15287, 11623, 7899]


#### Notice that the first 16 printed values of each row is the same (41344)
Since 43234 is the number of match data entries in dataset, this means that all the match entries in the source data did reach 16 rounds, which fulfils our 1st Criteria of counting the match as valid. As such, no cleaning is required to eliminate any invalid matches.

### Condition 2
Ensuring that every round in a match does not have any team spending <1000 or >44500

In [ ]:
#Cleaning datacopy
#Filling in blanks (games did not last till x rounds) with 0
#Removing any invalid games (Set them as np.nan to be dropped by dropna function)

for i in range(1, 31): #Iterating through 30 rounds
  winner = "_winner" 
  t1 = "_t1" 
  t2 = "_t2" 
  roundwinner = str(i) + winner
  team1 = str(i) + t1
  team2 = str(i) + t2
  t1column = datacopy[team1] #how much t1 spent on round i
  t2column = datacopy[team2] #how much t2 spent on round i
  winnercolumn = datacopy[roundwinner] #winner of i round

  for j in range(len(datacopy.index)):
    if t1column[j] < 1000 or t1column[j] > 44500: #Invalid spending from team 1
      datacopy[team1][j] = np.nan 
    if t2column[j] < 1000 or t2column[j] > 44500: #Invalid spending from team 2
      datacopy[team2][j] = np.nan
    if winnercolumn[j] != 1 and winnercolumn[j] != 2: #Games did not last till i rounds, fill all blanks with 0 and spending 5 (Preventing dropna from removing match entries that end before Round 30)
      datacopy[roundwinner][j] = 0 
      datacopy[team2][j] = 0
      datacopy[team1][j] = 5

In [84]:
clean = datacopy.dropna(axis = 0, how = "any") #Clean is a dataframe that contains all matches that meet Condition 1 of our Data Cleaning

In [85]:
clean.head()

,date,match_id,event_id,team_1,team_2,best_of,_map,t1_start,t2_start,1_t1,...,21_winner,22_winner,23_winner,24_winner,25_winner,26_winner,27_winner,28_winner,29_winner,30_winner
0,2020-03-01,2339402,4901,G2,Natus Vincere,5,Nuke,t,ct,4350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-01,2339402,4901,G2,Natus Vincere,5,Dust2,ct,t,3900.0,...,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,0.0
2,2020-03-01,2339402,4901,G2,Natus Vincere,5,Mirage,t,ct,4150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Dust2,t,ct,4150.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Nuke,ct,t,4200.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
clean.shape

(41344, 99)

#### Notice that the clean dataframe only has 41344 match entries

This means that 1890 matches has been disqualified and removed for not meeting Condition 2.

### Condition 3
Ensuring that every match entry is played on a valid map (within the 9 maps of the competitive map pool)

In [87]:
mapColumn = clean["_map"] #Extract all the maps played for the matches in clean
mapsList = mapColumn.tolist() #Convert the 
mapsList = dict.fromkeys(mapsList)
print(mapsList)

{'Nuke': None, 'Dust2': None, 'Mirage': None, 'Inferno': None, 'Train': None, 'Vertigo': None, 'Overpass': None, 'Default': None, 'Cobblestone': None, 'Cache': None}


From the Map List, you can observe that the 9 maps (Cache, Vertigo, Inferno, Dust 2, Mirage, Nuke, Overpass, Train, Cobblestone) are there. However, default is also listed. To remove default, we employ the following code

In [88]:
clean = clean[clean["_map"] != "Default"]

In [89]:
clean.head()

,date,match_id,event_id,team_1,team_2,best_of,_map,t1_start,t2_start,1_t1,...,21_winner,22_winner,23_winner,24_winner,25_winner,26_winner,27_winner,28_winner,29_winner,30_winner
0,2020-03-01,2339402,4901,G2,Natus Vincere,5,Nuke,t,ct,4350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-01,2339402,4901,G2,Natus Vincere,5,Dust2,ct,t,3900.0,...,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,0.0
2,2020-03-01,2339402,4901,G2,Natus Vincere,5,Mirage,t,ct,4150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Dust2,t,ct,4150.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Nuke,ct,t,4200.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
clean.shape

(41160, 99)

#### Notice that the clean dataframe now only has 41160 entries

This means that 184 entries have been removed as the map is Default, which is not one of the 9 competitive maps

cleanCopy = clean## Outliers

To calculate the outliers in our matches, we will first calculate the advantage that a team has over their opposition for each individual round.

In [91]:
cleanCopy = clean

In [92]:
#Create columns in dataframe that represent Team 1's advantage in economic spending for all 30 rounds
#This advantage is equal to Spending by Team 1 - Spending by Team 2 for said round

for i in range(1, 31):
  advantage_string = str(i) + "_advantage"
  t1_string = str(i) + "_t1"
  t2_string = str(i) + "_t2"
  
  advantage = cleanCopy[t1_string] - cleanCopy[t2_string]
  cleanCopy[advantage_string] = advantage
    
  
cleanCopy.head()

,date,match_id,event_id,team_1,team_2,best_of,_map,t1_start,t2_start,1_t1,...,21_advantage,22_advantage,23_advantage,24_advantage,25_advantage,26_advantage,27_advantage,28_advantage,29_advantage,30_advantage
0,2020-03-01,2339402,4901,G2,Natus Vincere,5,Nuke,t,ct,4350.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1,2020-03-01,2339402,4901,G2,Natus Vincere,5,Dust2,ct,t,3900.0,...,-26000.0,-50.0,-5550.0,4900.0,23950.0,1100.0,600.0,-1250.0,-2800.0,5.0
2,2020-03-01,2339402,4901,G2,Natus Vincere,5,Mirage,t,ct,4150.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
3,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Dust2,t,ct,4150.0,...,-4500.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
4,2020-02-29,2339401,4901,Natus Vincere,Astralis,3,Nuke,ct,t,4200.0,...,13750.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


Now we will create a dataframe with only the advantage and result of the round. Note that a positive advantage refers to an advantage for Team 1 and a negative advantage, an advantage for team 2.

In [93]:
cleanOutliers = pd.DataFrame(columns = ['Advantage', 'Result'])

for i in range(1,31):
  advantageString  = str(i) + "_advantage"
  winnerString = str(i) + "_winner"

  
  tempFrame = pd.concat([cleanCopy[advantageString], cleanCopy[winnerString]], ignore_index = True, axis = 1)
  tempFrame.rename(columns = {0: 'Advantage', 1: 'Result'}, inplace = True)

  cleanOutliers = pd.concat([cleanOutliers, tempFrame], axis = 0, ignore_index = True)

cleanOutliers.head()

,Advantage,Result
0,100.0,2.0
1,400.0,2.0
2,-50.0,2.0
3,-100.0,1.0
4,0.0,2.0


In [94]:
cleanOutliers.shape

(1234800, 2)

In [95]:
def not_outlier(x):
    iqr = x.quantile(0.75) - x.quantile(0.25)
    uppFence = x.quantile(0.75) + 1.5 * iqr
    lowFence = x.quantile(0.25) - 1.5 * iqr
    u = max(x[x<uppFence])
    l = min(x[x>lowFence])
    
    return [u,l]

#Function that returns upper and lower bounds of values considered not to be an outlier

In [96]:
notOutliers = pd.DataFrame(columns = ['Advantage','Result'])

fence = not_outlier(cleanOutliers['Advantage'])

In [ ]:
notOutliers = cleanOutliers[cleanOutliers['Advantage'].between(fence[1], fence[0],inclusive = True)]

#Function that eliminates all entries with Advantage that are outliers

In [98]:
notOutliers.shape

(1128333, 2)

From the above number, you can identify that there are about 106467 rounds that have an Advantage that can be considered an Outlier. However, we chose to not remove it as this "Outlier" advantages should just mean that the Team with the advantage should in theory, win the round convincingly 

# Conclusion

After our Data Cleaning, we have removed 2074 match entries for not meeting our 3 conditions to qualify them as a valid match. We will now export this cleaned dataframe into a csv for use in the other sections of this notebook

In [99]:
clean.to_csv('/data/clean.csv', index = False)